### FOR Paper Figures

In [9]:
# ######################################## Transmission ########################################
# # select the protocol for each layer
# PHYSICAL_PROTOCOLS = "Generic_physical" # set the protocol for the physical layer
# DATALINK_PROTOCOLS = "Generic_datalink" # set the protocol for the datalink layer
# NETWORK_PROTOCOLS = "Generic_network" # set the protocol for the network layer
# TRANSPORT_PROTOCOLS = "Generic_transport" # set the protocol for the transport layer
# SESSION_PROTOCOLS = "Generic_session" # set the protocol for the session layer
# PRESENTATION_PROTOCOLS = "Generic_presentation" # set the protocol for the presentation layer
# APPLICATION_PROTOCOLS = "Generic_application" # set the protocol for the application layer

# FAILURE_RATE = 0.0 # Select transmission failure rate causing retransmission given as a float between 0 and 1


# ######################################## Data Preprocessing ########################################
# PREPROCESSING_TYPE = "normalization" # set the preprocessing to apply to the data options: normalization, min_max_scaling, GADF



# ######################################## Training ########################################
# MODEL_NAME ="SimpleCNN" # set which model to use examples: KAN, resnet18, baichuan-inc/Baichuan-13B-Chat
# NUM_EPOCHS = 50
# BATCH_SIZE = 32
# # INPUT_SIZE = (1, 3, 224, 224) # 4d input for resnet # (1,128) # batch, max_seq_length for llm
# if MODEL_NAME == "SimpleCNN":
#     INPUT_SIZE = (1, 1,10) # 4d input for resnet # (1,128) # batch, max_seq_length for llm
# else:
#     INPUT_SIZE = (1, 10) # 

# EVALUATION_STRATEGY = "cross_validation" # set the evaluation strategy options: cross_validation, train_test_split
# K_FOLDS = 5 # Only used if EVALUATION_STRATEGY is cross_validation
# SPLIT_RATIO = 0.8 # Only used if EVALUATION_STRATEGY is train_test_split



# NUM_LAYERS = 2
# GRID_SIZE = 10
# NUM_CLASSES = 2
# DIN = [10,10,10]
# DOUT = 2

# # Transformer specific parameters
# CONTEXT_LENGTH = 10  
# EMBEDDING_SIZE = 16  
# NUM_HEADS = 2       
# NUM_DECODER_BLOCKS = 3  
# FEED_FORWARD_SIZE = 64  

# ######################################## Inference ########################################
# NUM_INFERENCES = 10000 # number of inferences to run


# # GENERAL CONFIG
# NUM_SAMPLES = 256 # number of samples that are used to calculate the energy consumption
# FLOAT_PRECISION = 32 # number of bits used to represent a floating point number
# PROCESSOR_FLOPS_PER_SECOND = 1e12 # theoretical maximum number of floating point operations per second for the processor
# PROCESSOR_MAX_POWER = 100 # maximum power consumption of the processor in Watts

# # Timeseries specific
# SAMPLE_SIZE = 10 # size of a single sample e.g. number of timesteps in a timeseries or number of pixels in an image


### Actual calculator

In [ ]:
import sys
from pathlib import Path

root = Path().resolve().parent  # go up one level from notebook’s folder
sys.path.append(str(root))

import pandas as pd
from calculators.Training import Training
from calculators.ModelFLOPS import MLPCalculator, CNNCalculator, KANCalculator, TransformerCalculator



######################################## Data Preprocessing ########################################
PREPROCESSING_TYPE = "normalization" # set the preprocessing to apply to the data options: normalization, min_max_scaling, GADF

REAL_INPUT_SIZE = 10 ## CHANGE SIZE HERE

######################################## Training ########################################
MODEL_NAME ="SimpleCNN" # set which model to use examples: KAN, resnet18, baichuan-inc/Baichuan-13B-Chat
NUM_EPOCHS = 50
BATCH_SIZE = 32
# INPUT_SIZE = (1, 3, 224, 224) # 4d input for resnet # (1,128) # batch, max_seq_length for llm
if MODEL_NAME == "SimpleCNN":
    INPUT_SIZE = (1, 1, REAL_INPUT_SIZE) # 4d input for resnet # (1,128) # batch, max_seq_length for llm
else:
    INPUT_SIZE = (1, REAL_INPUT_SIZE) # 

EVALUATION_STRATEGY = "cross_validation" # set the evaluation strategy options: cross_validation, train_test_split
K_FOLDS = 5 # Only used if EVALUATION_STRATEGY is cross_validation
SPLIT_RATIO = 0.8 # Only used if EVALUATION_STRATEGY is train_test_split

NUM_LAYERS = 2
GRID_SIZE = 5
NUM_CLASSES = 2
DIN = REAL_INPUT_SIZE
DOUT = 2

# Transformer specific parameters
CONTEXT_LENGTH = REAL_INPUT_SIZE  
EMBEDDING_SIZE = 16  
NUM_HEADS = 2       
NUM_DECODER_BLOCKS = 3  
FEED_FORWARD_SIZE = 64  

######################################## Inference ########################################
NUM_INFERENCES = 10000 # number of inferences to run


# GENERAL CONFIG
NUM_SAMPLES = 256 # number of samples that are used to calculate the energy consumption
FLOAT_PRECISION = 32 # number of bits used to represent a floating point number
PROCESSOR_FLOPS_PER_SECOND = 1e12 # theoretical maximum number of floating point operations per second for the processor
PROCESSOR_MAX_POWER = 100 # maximum power consumption of the processor in Watts

# Timeseries specific
SAMPLE_SIZE = REAL_INPUT_SIZE # size of a single sample e.g. number of timesteps in a timeseries or number of pixels in an image


NUM_LAYERS = [3,4,5,6,7] # K
HIDDEN_SIZE = [10] # H
energy_consumption  ={}
model_names = ['MLP', 'CNN' ,'KAN', 'Transformer']
calculator = None
for model_name in model_names:
    curr_model = {}
    for L in NUM_LAYERS:
        for hs in HIDDEN_SIZE:
            #INPUT_SIZE = (1, 1, 10) #is this ok for all models?
            print(f"Calculating energy consumption for {model_name} with {L} layers and hidden size {hs}")
            calculator = None
            if model_name == "MLP":
                calculator = MLPCalculator(
                    num_layers=L,
                    num_classes=NUM_CLASSES,
                    din=REAL_INPUT_SIZE,
                    dout=DOUT,
                    num_samples=SAMPLE_SIZE  # for time series

                )
            elif model_name == "CNN":
                calculator = CNNCalculator(
                    num_cnv_layers=L,
                    num_pool_layers=L,
                    i_r=10,
                    i_c=1,
                    k_r=3,
                    k_c=1,
                    c_in=1,
                    num_classes=NUM_CLASSES,
                    #din=DIN,
                    #dout=DOUT,
                    num_samples=SAMPLE_SIZE  # for time series

                )
            elif model_name == "KAN":
                hs_all =[SAMPLE_SIZE] +[hs]*L
                calculator = KANCalculator(
                    num_layers=L,
                    grid_size=hs,
                    num_classes=NUM_CLASSES,
                    din=10,
                    dout=DOUT,
                    num_samples=REAL_INPUT_SIZE # for time series
                )
            elif model_name == "Transformer":  # Changed from "SimpleTransformer" to "Transformer"
                calculator = TransformerCalculator(
                    context_length=CONTEXT_LENGTH,
                    embedding_size=hs,
                    num_heads=NUM_HEADS,
                    num_decoder_blocks=L,
                    feed_forward_size=FEED_FORWARD_SIZE,
                    vocab_size=20
                )
           
            training = Training(
                model_name=model_name,
                num_epochs=NUM_EPOCHS,
                batch_size=BATCH_SIZE,
                processor_flops_per_second=PROCESSOR_FLOPS_PER_SECOND,
                processor_max_power=PROCESSOR_MAX_POWER,
                num_samples=NUM_SAMPLES,
                input_size=INPUT_SIZE,
                evaluation_strategy=EVALUATION_STRATEGY,
                k_folds=K_FOLDS,
                split_ratio=SPLIT_RATIO,
                calculator=calculator
            )
            if model_name == "MLP":
                model_str = "MLP"
            elif model_name == "CNN":
                model_str = "CNN"
            elif model_name == "Transformer":
                model_str = "Transformer"
            else:
                model_str = model_name

            curr_model[(L,hs)] = training.calculate_energy()['training_energy']
    energy_consumption[model_str] = curr_model


import pandas as pd
df = pd.DataFrame(energy_consumption)
df.to_csv('energy_consumption_models_small_theor.csv')
df

Calculating energy consumption for MLP with 3 layers and hidden size 10
Calculating energy consumption for MLP with 4 layers and hidden size 10
Calculating energy consumption for MLP with 5 layers and hidden size 10
Calculating energy consumption for MLP with 6 layers and hidden size 10
Calculating energy consumption for MLP with 7 layers and hidden size 10
Calculating energy consumption for CNN with 3 layers and hidden size 10
Calculating energy consumption for CNN with 4 layers and hidden size 10
Calculating energy consumption for CNN with 5 layers and hidden size 10
Calculating energy consumption for CNN with 6 layers and hidden size 10
Calculating energy consumption for CNN with 7 layers and hidden size 10
Calculating energy consumption for KAN with 3 layers and hidden size 10
Calculating energy consumption for KAN with 4 layers and hidden size 10
Calculating energy consumption for KAN with 5 layers and hidden size 10
Calculating energy consumption for KAN with 6 layers and hidden 

,,MLP,CNN,KAN,Transformer
3,10,0.006758,0.000614,1.250918,1.732608
4,10,0.010138,0.000614,1.876378,2.310144
5,10,0.013517,0.000614,2.501837,2.887680
6,10,0.016896,0.000614,3.127296,3.465216
7,10,0.020275,0.000614,3.752755,4.042752
